# Trial table simple

We create a simple data frame with the start and end time for each trial.



In [1]:
%load_ext autoreload
%autoreload 2
%run setup_project.py

Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 40 testing sessions in the list
See myProject and sSesList objects


In [2]:
def prepareAutopipySession(ses):
    """
    Create a data frame for autopi trial times (only ROS time)
    
    Arguments:
    ses: autopipy session
    
    Results:
    Save a .trials object in ses
    """
    ses.segmentTrialsFromLog(verbose=True) # for more details, set verbose to True
    ses.createTrialElectroList() # create a List of trial object
    ses.loadLogFile()
    for t in ses.trialList:
        t.getLightCondition(ses.log)
    lightCondition = [ t.light  for t in ses.trialList]
    ses.trials["light"]=lightCondition
    
def prepareSpikeASession(sSes):
    """
    Load the Animal_pose for a spikeA session, save ap in the sSes object
    """
    sSes.load_parameters_from_files() 
    ap = Animal_pose(sSes)
    ap.load_pose_from_file()
    sSes.ap = ap
    
def electroTimeToTrial(ses,sSes):
    """
    Add the electrophysiology time to the ses.trials data frame and save it to a file
    """
    rosToEle = interp1d(sSes.ap.pose[:,7],sSes.ap.pose[:,0],bounds_error=False)
    print(ses.trials.shape)
    ses.trials["recStartTime"] = rosToEle(ses.trials.startTime)
    ses.trials["recEndTime"] = rosToEle(ses.trials.endTime)
    ses.trials = ses.trials.dropna()
    print(ses.trials.shape)
    ses.trials = ses.trials[ses.trials.recEndTime>ses.trials.recStartTime] # remove any negative duration
    print(ses.trials.shape)
    fn = ses.path+"/"+"trial_table_simple"
    print("Saving " + fn)
    ses.trials.to_csv(fn,index=False)

    

In [3]:
for i,(ses,sSes) in enumerate(zip(myProject.sessionList,sSesList)):
    print(i,ses.name)
    prepareAutopipySession(ses)
    prepareSpikeASession(sSes)
    electroTimeToTrial(ses,sSes)

0 mn5824-20112020-0107
mn5824-20112020-0107 trial segmentation
Remove last opening of the door
Number of trials : 152
(152, 7)
(150, 9)
(150, 9)
Saving /adata/projects/autopi_ca1/mn5824/mn5824-20112020-0107/trial_table_simple
1 mn5824-22112020-0107
mn5824-22112020-0107 trial segmentation
Number of trials : 124
(124, 7)
(124, 9)
(124, 9)
Saving /adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107/trial_table_simple
2 mn5824-24112020-0107
mn5824-24112020-0107 trial segmentation
Remove last opening of the door
Number of trials : 135
(135, 7)
(135, 9)
(135, 9)
Saving /adata/projects/autopi_ca1/mn5824/mn5824-24112020-0107/trial_table_simple
3 mn5824-02122020-0106
mn5824-02122020-0106 trial segmentation
problem with the door alternation
Problem with index 5497
Removing the second of two door closings (index :5497)
Number of trials : 156
(156, 7)
(156, 9)
(156, 9)
Saving /adata/projects/autopi_ca1/mn5824/mn5824-02122020-0106/trial_table_simple
4 mn711-28012021-0106
mn711-28012021-0106 trial